In [198]:
import sys
from pathlib import Path

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

# Verify path
print(f"Python will look in these locations:\n{sys.path}")


# # --- Execute the processor ---
# import utils
# from config import date_str, DOWNLOAD_DIR, DEST_DIR

date_str = '2025-04-17'
print(f'\nSTOCK SELECTION DATE: {date_str}')



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Python will look in these locations:
['C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\python311.zip', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\DLLs', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\Lib', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv', '', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\ping\\Files_win10\\python\\py311\\stocks\\src', 'c:\\Users\\ping\\Files_win10\\python\\py311\\stocks\\src', 'c:\\Users\\ping\\Files_win10\\python\\py311\\stocks\\src', 'c:\\Users\\ping\\Files_win10\\python\\py311\\stocks\\src', 

In [199]:
import pandas as pd
# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
pd.set_option('display.width', 1000)        # Let the display adjust to the window
# pd.set_option('display.max_colwidth', None) # Show full content of each cell
pd.set_option('display.max_rows', 200)
# pd.set_option('display.width', 120)
pd.set_option('display.float_format', '{:.4f}'.format)

In [200]:
zscore_df = pd.read_parquet(f'..\data\{date_str}_zscore_df.parquet', engine='pyarrow')
cluster_stats_df = pd.read_parquet(f'..\data\{date_str}_cluster_stats_df.parquet', engine='pyarrow')
detailed_clusters_df = pd.read_parquet(f'..\data\{date_str}_detailed_clusters_df.parquet', engine='pyarrow')
# df_finviz_merged = pd.read_parquet(f'..\data\{date_str}_df_finviz_merged.parquet', engine='pyarrow')

In [201]:
detailed_clusters_df

,Cluster_ID,Ticker,Raw_Score,Risk_Adj_Score,Volatility,Price,"MktCap AUM, M"
147,1,FLOT,0.7490,186.3888,0.0040,50.7600,8800.0000
33,1,LNG,1.3665,44.2172,0.0309,231.4500,51570.0000
99,1,EQT,0.8919,29.0666,0.0307,50.9800,30460.0000
141,1,EXE,0.7577,28.9311,0.0262,106.4900,24780.0000
188,1,WMB,0.6206,27.4544,0.0226,58.6200,71560.0000
...,...,...,...,...,...,...,...
206,60,BNS,0.5713,32.7277,0.0175,47.4700,59130.0000
273,60,DEO,0.4458,22.8907,0.0195,109.5800,60880.0000
535,60,ELS,0.0029,0.1779,0.0165,65.3200,13080.0000
540,60,HR,-0.0061,-0.3265,0.0186,16.0000,5620.0000


In [202]:
# df_finviz_merged = pd.read_parquet(f'..\data\{date_str}_df_finviz_merged.parquet', engine='pyarrow')
df_OHLCV = pd.read_parquet(r'..\data\2025-04-17_df_OHLCV_clean.parquet', engine='pyarrow')
df_OHLCV

Open     High      Low    Close  Adj Close     Volume  Adj Open  Adj High  Adj Low
Symbol Date                                                                                             
AAPL   2025-04-17 197.1500 198.8300 194.4200 196.9800   196.9800   50423277  197.1500  198.8300 194.4200
       2025-04-16 198.3600 200.7000 192.3700 194.2700   194.2700   59732400  198.3600  200.7000 192.3700
       2025-04-15 201.8600 203.5100 199.8000 202.1400   202.1400   51343900  201.8600  203.5100 199.8000
       2025-04-14 211.4400 212.9400 201.1600 202.5200   202.5200  101352900  211.4400  212.9400 201.1600
       2025-04-11 186.1000 199.5400 186.0600 198.1500   198.1500   87435900  186.1000  199.5400 186.0600
...                    ...      ...      ...      ...        ...        ...       ...       ...      ...
ARGX   2024-04-24 380.8500 380.8500 372.6800 375.0000   375.0000     266400  380.8500  380.8500 372.6800
       2024-04-23 368.6900 377.9400 368.6900 375.0800   375.0800     340200  368.6900  377.9400 368.6900
       2024-04-22 362.9300 370.8000 359.8300 368.7500   368.7500     335800  362.9300  370.8000 359.8300
       2024-04-19 360.3800 362.5400 357.5500 360.2300   360.2300     329100  360.3800  362.5400 357.5500
       2024-04-18 362.3400 367.4900 358.0900 358.4100   358.4100     261800  362.3400  367.4900 358.0900

[386038 rows x 9 columns]

In [ ]:
import pandas as pd # Assuming pandas is used

def select_stocks_from_clusters(cluster_stats_df, detailed_clusters_df,
                                select_top_n_clusters=3, max_selection_per_cluster=5,
                                min_cluster_size=5, penalty_IntraCluster_Corr=0.3,
                                date_str=date_str,
                                min_raw_score=None, # <-- Added argument
                                min_risk_adj_score=None): # <-- Added argument
    """
    Pipeline to select stocks from better performing clusters, with optional score thresholds.

    Parameters:
    - cluster_stats_df: DataFrame with cluster statistics.
    - detailed_clusters_df: DataFrame with detailed cluster information including
                            'Ticker', 'Cluster_ID', 'Raw_Score', 'Risk_Adj_Score', etc.
    - select_top_n_clusters: int, Number of top clusters to select (default=3).
    - max_selection_per_cluster: int, Max number of stocks to select from each cluster (default=5).
    - min_cluster_size: int, Minimum size for a cluster to be considered (default=5).
    - penalty_IntraCluster_Corr: float, Penalty weight for intra-cluster correlation in
                                     composite score (default=0.3).
    - date_str: str, Date string for tracking/parameter storage.
    - min_raw_score: float, optional (default=None)
        Minimum Raw_Score required for a stock to be considered for selection.
        If None, no threshold is applied based on Raw_Score.
    - min_risk_adj_score: float, optional (default=None)
        Minimum Risk_Adj_Score required for a stock to be considered for selection.
        If None, no threshold is applied based on Risk_Adj_Score.

    Returns:
    - dict: A dictionary containing:
        - 'selected_top_n_cluster_ids': List of top selected cluster IDs.
        - 'selected_stocks': DataFrame of selected stocks.
        - 'cluster_performance': DataFrame of selected cluster metrics.
        - 'parameters': Dictionary of the input parameters used.
    """

    # Store input parameters
    parameters = {
        'date_str': date_str,
        'select_top_n_clusters': select_top_n_clusters,
        'max_selection_per_cluster': max_selection_per_cluster,
        'min_cluster_size': min_cluster_size,
        'min_raw_score': min_raw_score,         # <-- Stored parameter
        'min_risk_adj_score': min_risk_adj_score, # <-- Stored parameter
        'penalty_IntraCluster_Corr': penalty_IntraCluster_Corr,
    }
    
    # ===== 1. Filter and Rank Clusters =====
    qualified_clusters = cluster_stats_df[cluster_stats_df['Size'] >= min_cluster_size].copy()
    if qualified_clusters.empty:
        print(f"Warning: No clusters met the minimum size criteria ({min_cluster_size}).")
        return {
            'selected_stocks': pd.DataFrame(),
            'cluster_performance': pd.DataFrame(),
            'parameters': parameters
        }

    qualified_clusters['Composite_Cluster_Score'] = (
        (1 - penalty_IntraCluster_Corr) * qualified_clusters['Avg_Raw_Score'] +
        penalty_IntraCluster_Corr * (1 - qualified_clusters['Avg_IntraCluster_Corr'])
    )
    ranked_clusters = qualified_clusters.sort_values('Composite_Cluster_Score', ascending=False)
    selected_clusters = ranked_clusters.head(select_top_n_clusters)
    cluster_ids = selected_clusters['Cluster_ID'].tolist()

    if not cluster_ids:
        print("Warning: No clusters were selected based on ranking.")
        return {
            'selected_stocks': pd.DataFrame(),
            'cluster_performance': selected_clusters, # Return empty selected clusters df
            'parameters': parameters
        }


    # ===== 2. Select Stocks from Each Cluster =====
    selected_stocks_list = []
    for cluster_id in cluster_ids:
        # Get all stocks for the current cluster
        cluster_stocks = detailed_clusters_df[detailed_clusters_df['Cluster_ID'] == cluster_id].copy()

        # ===> Apply Threshold Filters <===
        if min_raw_score is not None:
            cluster_stocks = cluster_stocks[cluster_stocks['Raw_Score'] >= min_raw_score]
        if min_risk_adj_score is not None:
            cluster_stocks = cluster_stocks[cluster_stocks['Risk_Adj_Score'] >= min_risk_adj_score]
        # ===> End of Added Filters <===

        # Proceed only if stocks remain after filtering
        if len(cluster_stocks) > 0:
            # Sort remaining stocks by Risk_Adj_Score and select top N
            top_stocks = cluster_stocks.sort_values('Risk_Adj_Score', ascending=False).head(max_selection_per_cluster)

            # Add cluster-level metrics to the selected stock rows
            cluster_metrics = selected_clusters[selected_clusters['Cluster_ID'] == cluster_id].iloc[0]
            for col in ['Composite_Cluster_Score', 'Avg_IntraCluster_Corr', 'Avg_Volatility',
                      'Avg_Raw_Score', 'Avg_Risk_Adj_Score', 'Size']: # Added Size for context
                # Use .get() for safety if a column might be missing
                top_stocks[f'Cluster_{col}'] = cluster_metrics.get(col, None)
            selected_stocks_list.append(top_stocks)

    # Consolidate selected stocks
    if selected_stocks_list:
        selected_stocks = pd.concat(selected_stocks_list)
        # Recalculate weights based on the final selection
        if selected_stocks['Minimum Variance or Risk Parity weighting '].sum() != 0:
             selected_stocks['Weight'] = (selected_stocks['Risk_Adj_Score'] /
                                          selected_stocks['Risk_Adj_Score'].sum())
        else:
             # Handle case where all selected scores are zero (unlikely but possible)
             selected_stocks['Weight'] = 1 / len(selected_stocks) if len(selected_stocks) > 0 else 0

        selected_stocks = selected_stocks.sort_values(['Cluster_ID', 'Risk_Adj_Score'],
                                                    ascending=[True, False])
    else:
        selected_stocks = pd.DataFrame()
        print("Warning: No stocks met selection criteria (including score thresholds if applied).")


    # ===== 3. Prepare Enhanced Output Reports =====
    cluster_performance = selected_clusters.copy()
    # Calculate how many stocks were actually selected per cluster after filtering
    cluster_performance['Stocks_Selected'] = cluster_performance['Cluster_ID'].apply(
        lambda x: len(selected_stocks[selected_stocks['Cluster_ID'] == x]) if not selected_stocks.empty else 0)

    if not selected_stocks.empty:
         # Ensure Avg_IntraCluster_Corr exists before calculating diversification
        if 'Avg_IntraCluster_Corr' in cluster_performance.columns:
             cluster_performance['Intra_Cluster_Diversification'] = 1 - cluster_performance['Avg_IntraCluster_Corr']
        else:
             cluster_performance['Intra_Cluster_Diversification'] = pd.NA # Or None
    else:
      # Handle case where selected_stocks is empty
        cluster_performance['Intra_Cluster_Diversification'] = pd.NA # Or None

    # ===> Package results and parameters
    results_bundle = {
        'selected_top_n_cluster_ids': cluster_ids,
        'selected_stocks': selected_stocks,
        'cluster_performance': cluster_performance,
        'parameters': parameters
    }

    return results_bundle


In [204]:
import pandas as pd
from typing import Dict, Any

def print_stock_selection_report(output: Dict[str, Any]) -> None:
    """
    Prints a detailed report summarizing the results of the stock selection process,
    extracting all necessary information from the output dictionary.

    Args:
        output (Dict[str, Any]): The dictionary returned by the
                                 select_stocks_from_clusters function, containing:
                                 - 'selected_stocks': DataFrame of selected stocks.
                                 - 'cluster_performance': DataFrame of selected cluster metrics.
                                #  - 'parameters': Dictionary of the input parameters used.
                                #  - 'cluster_stats_df': Original cluster stats DataFrame.
                                #  - 'detailed_clusters_df': Original detailed clusters DataFrame.
    Returns:
        None: This function prints output to the console.
    """
    # Extract data from the output dictionary using .get() for safety
    selected_stocks = output.get('selected_stocks', pd.DataFrame())
    cluster_performance = output.get('cluster_performance', pd.DataFrame())
    used_params = output.get('parameters', {})
    # Extract the input DataFrames needed for the report
    # cluster_stats_df = output.get('input_cluster_stats_df') # Might be None
    cluster_stats_df = output.get('cluster_stats_df') # Might be None
    # detailed_clusters_df = output.get('input_detailed_clusters_df') # Might be None
    detailed_clusters_df = output.get('detailed_clusters_df') # Might be None

    # --- Start of Original Code Block (adapted) ---

    print("\n=== CLUSTER SELECTION CRITERIA ===")
    print("* Using Composite_Cluster_Score (balancing Raw Score and diversification) for cluster ranking.")
    print("* Using Risk_Adj_Score for stock selection within clusters.")

    num_selected_clusters = len(cluster_performance) if not cluster_performance.empty else 0
    # Use the extracted cluster_stats_df
    total_clusters = len(cluster_stats_df) if cluster_stats_df is not None and not cluster_stats_df.empty else 'N/A'

    print(f"* Selected top {num_selected_clusters} clusters from {total_clusters} total initial clusters.") # Adjusted wording slightly
    print(f"* Selection Criteria:")
    if used_params:
        for key, value in used_params.items():
            # Avoid printing the large input dataframes stored in parameters if they were added there too
            if not isinstance(value, pd.DataFrame):
                 print(f"    {key}: {value}")
    else:
        print("    Parameters not available.")


    if not cluster_performance.empty:
        print("\n=== SELECTED CLUSTERS (RANKED BY COMPOSITE SCORE) ===")
        display_cols_exist = [col for col in [
                                'Cluster_ID', 'Size', 'Avg_Raw_Score', 'Avg_Risk_Adj_Score',
                                'Avg_IntraCluster_Corr', 'Avg_Volatility', 'Composite_Cluster_Score',
                                'Stocks_Selected', 'Intra_Cluster_Diversification']
                              if col in cluster_performance.columns]
        print(cluster_performance[display_cols_exist].sort_values('Composite_Cluster_Score', ascending=False).to_string(index=False))

        # Print top 8 stocks by Raw_Score for each selected cluster
        # Check if detailed_clusters_df was successfully extracted
        if detailed_clusters_df is not None and not detailed_clusters_df.empty:
            print("\n=== TOP STOCKS BY RAW SCORE PER SELECTED CLUSTER ===")
            print("""* Volatility is the standard deviation of daily returns over the past 250 trading days (example context).
* Note: The stocks below are shown ranked by Raw_Score for analysis,
*       but actual selection within the cluster was based on Risk_Adj_Score.""")

            for cluster_id in cluster_performance['Cluster_ID']:
                 cluster_stocks = detailed_clusters_df[detailed_clusters_df['Cluster_ID'] == cluster_id]
                 if not cluster_stocks.empty:
                    required_cols = ['Ticker', 'Raw_Score', 'Risk_Adj_Score', 'Volatility']
                    if all(col in cluster_stocks.columns for col in required_cols):
                        top_raw = cluster_stocks.nlargest(8, 'Raw_Score')[required_cols]

                        print(f"\nCluster {cluster_id} - Top 8 by Raw Score:")
                        print(top_raw.to_string(index=False))
                        cluster_avg_raw = cluster_performance.loc[cluster_performance['Cluster_ID'] == cluster_id, 'Avg_Raw_Score'].values
                        cluster_avg_risk = cluster_performance.loc[cluster_performance['Cluster_ID'] == cluster_id, 'Avg_Risk_Adj_Score'].values
                        if len(cluster_avg_raw) > 0: print(f"Cluster Avg Raw Score: {cluster_avg_raw[0]:.2f}")
                        if len(cluster_avg_risk) > 0: print(f"Cluster Avg Risk Adj Score: {cluster_avg_risk[0]:.2f}")
                    else:
                        print(f"\nCluster {cluster_id} - Missing required columns in detailed_clusters_df to show top stocks.")
                 else:
                     print(f"\nCluster {cluster_id} - No stocks found in detailed_clusters_df for this cluster.")
        else:
            print("\n=== TOP STOCKS BY RAW SCORE PER SELECTED CLUSTER ===")
            print("Skipping - Detailed cluster information ('input_detailed_clusters_df') not found in the output dictionary.")

    else:
        print("\n=== SELECTED CLUSTERS ===")
        print("No clusters were selected based on the criteria.")


    print(f"\n=== FINAL SELECTED STOCKS (FILTERED & WEIGHTED) ===")
    if not selected_stocks.empty:
        print("* Stocks actually selected based on Risk_Adj_Score (and optional thresholds) within each cluster.")
        print("* Position weights assigned based on Risk_Adj_Score within the final selected portfolio.")

        desired_cols = ['Cluster_ID', 'Ticker', 'Raw_Score', 'Risk_Adj_Score',
                        'Volatility', 'Weight',
                        'Cluster_Avg_Raw_Score', 'Cluster_Avg_Risk_Adj_Score']
        available_cols = [col for col in desired_cols if col in selected_stocks.columns]
        print(selected_stocks[available_cols].sort_values(['Cluster_ID', 'Risk_Adj_Score'],
                                                        ascending=[True, False]).to_string(index=False))

        print("\n=== PORTFOLIO SUMMARY ===")
        print(f"Total Stocks Selected: {len(selected_stocks)}")
        print(f"Average Raw Score: {selected_stocks.get('Raw_Score', pd.Series(dtype=float)).mean():.2f}")
        print(f"Average Risk-Adjusted Score: {selected_stocks.get('Risk_Adj_Score', pd.Series(dtype=float)).mean():.2f}")
        print(f"Average Volatility: {selected_stocks.get('Volatility', pd.Series(dtype=float)).mean():.2f}")
        print(f"Total Weight (should be close to 1.0): {selected_stocks.get('Weight', pd.Series(dtype=float)).sum():.4f}")
        print("\nCluster Distribution:")
        print(selected_stocks['Cluster_ID'].value_counts().to_string())
    else:
        print("No stocks were selected after applying all filters and criteria.")


In [205]:
import pandas as pd
import numpy as np
import itertools # Import the itertools module

# --- Define Factor Ranges ---
# Generate the factors using numpy.arange for float steps
# Add a small epsilon to the end value to ensure the endpoint is included due to float precision
raw_score_factors = np.arange(0.5, 1.2 + 0.01, 0.1)
risk_adj_score_factors = np.arange(0.5, 1.2 + 0.01, 0.1)
penalty_factors = np.arange(0, 0.4 + 0.01, 0.1) # New factor range

print("--- Parameter Ranges ---")
print(f"Raw Score Factors: {np.round(raw_score_factors,1)}")
print(f"Risk Adj Score Factors: {np.round(risk_adj_score_factors,1)}")
print(f"Penalty Factors: {np.round(penalty_factors,1)}")


# --- Generate All Combinations ---
# Use itertools.product to create an iterator of all combinations
parameter_combinations = list(itertools.product(raw_score_factors, risk_adj_score_factors, penalty_factors))
total_combinations = len(parameter_combinations)
print(f"\nTotal parameter combinations to iterate: {total_combinations}")


# --- Store results ---
all_portfolios = {} # Dictionary to store portfolios by name

# --- Fixed Parameters (that don't vary in this loop) ---
select_top_n_clusters = 60
max_selection_per_cluster = 2
min_cluster_size = 3  # prevent extreme high risk adj scores
# You might want to get the date dynamically
portf_date_base = date_str # Example date, adjust as needed (Ensure date_str is defined before this)

# --- Iteration Loop (Single Loop over Combinations) ---
print("\nStarting portfolio generation loop...")
for i, (raw_scale, risk_adj_scale, penalty) in enumerate(parameter_combinations):

    # --- Calculate dynamic parameters based on current factors ---
    # Round factors slightly to avoid potential floating point representation issues in calculations/names
    raw_scale = round(raw_scale, 1)
    risk_adj_scale = round(risk_adj_scale, 1)
    penalty = round(penalty, 1) # Round the new penalty factor as well

    min_raw_score = 2.0 * raw_scale
    min_risk_adj_score = 100.0 * risk_adj_scale
    # penalty_IntraCluster_Corr is now the 'penalty' variable from the combination

    print(f"\nRunning combination {i+1}/{total_combinations}: "
          f"raw_F={raw_scale:.1f}, riskAdj_F={risk_adj_scale:.1f}, penalty={penalty:.1f}")
    print(f"Resulting thresholds: min_raw_score={min_raw_score:.2f}, min_risk_adj_score={min_risk_adj_score:.1f}")

    # --- Run the selection pipeline ---
    try:
        output = select_stocks_from_clusters(
            cluster_stats_df=cluster_stats_df,      # Ensure this DataFrame is loaded/defined
            detailed_clusters_df=detailed_clusters_df, # Ensure this DataFrame is loaded/defined
            select_top_n_clusters=select_top_n_clusters,
            max_selection_per_cluster=max_selection_per_cluster,
            min_cluster_size=min_cluster_size,
            penalty_IntraCluster_Corr=penalty, # Use the penalty from the current combination
            min_raw_score=min_raw_score,
            min_risk_adj_score=min_risk_adj_score,
            date_str=portf_date_base # Pass the date
        )

        # --- Process and Store Results ---
        portf_date = output['parameters']['date_str']
        portf_raw_score_val = output['parameters']['min_raw_score'] # Use the value returned by the function
        portf_risk_adj_score_val = output['parameters']['min_risk_adj_score'] # Use the value returned by the function
        portf_penalty_val = output['parameters']['penalty_IntraCluster_Corr'] # Get penalty used
        _selected_stocks = output['selected_stocks']

        if _selected_stocks is not None and not _selected_stocks.empty:
            portf_selected_stocks = _selected_stocks.set_index('Ticker')[['Weight']] # Select only Weight column after setting index

            # Format the name using the factors - now includes penalty
            portf_name = f'{portf_date}_portf_rawF_{raw_scale:.1f}_riskAdjF_{risk_adj_scale:.1f}_pen_{penalty:.1f}'
            # Alternative using resulting thresholds and penalty:
            # portf_name = f'{portf_date}_portf_raw_{portf_raw_score_val:.2f}_riskadj_{portf_risk_adj_score_val:.1f}_pen_{portf_penalty_val:.1f}'

            print(f'Generated Portfolio:')
            print(f'  Name: {portf_name}')
            print(f'  Number of stocks: {len(portf_selected_stocks)}')
            # print(f'port_selected_stocks:\n{portf_selected_stocks}') # Can be verbose

            # Store the results - using the portfolio name as the key
            all_portfolios[portf_name] = {
                'parameters': {
                    'raw_score_scale_factor': raw_scale,
                    'risk_adj_score_scale_factor': risk_adj_scale,
                    'penalty_IntraCluster_Corr': penalty, # Store the penalty factor used
                    'min_raw_score': portf_raw_score_val,
                    'min_risk_adj_score': portf_risk_adj_score_val,
                    'select_top_n_clusters': select_top_n_clusters,
                    'max_selection_per_cluster': max_selection_per_cluster,
                    'min_cluster_size': min_cluster_size,
                    'date': portf_date
                },
                'selected_stocks': portf_selected_stocks
            }
        else:
             print(f"No stocks selected for raw_F={raw_scale:.1f}, riskAdj_F={risk_adj_scale:.1f}, penalty={penalty:.1f}")

    except Exception as e:
        print(f"ERROR processing combination raw_F={raw_scale:.1f}, riskAdj_F={risk_adj_scale:.1f}, penalty={penalty:.1f}: {e}")
        # Decide if you want to continue or stop on error - currently continues

print(f"\n--- Portfolio Generation Complete ---")
print(f"Generated {len(all_portfolios)} portfolios out of {total_combinations} combinations attempted.")



--- Parameter Ranges ---
Raw Score Factors: [0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2]
Risk Adj Score Factors: [0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2]
Penalty Factors: [0.  0.1 0.2 0.3 0.4]

Total parameter combinations to iterate: 320

Starting portfolio generation loop...

Running combination 1/320: raw_F=0.5, riskAdj_F=0.5, penalty=0.0
Resulting thresholds: min_raw_score=1.00, min_risk_adj_score=50.0
Generated Portfolio:
  Name: 2025-04-17_portf_rawF_0.5_riskAdjF_0.5_pen_0.0
  Number of stocks: 33

Running combination 2/320: raw_F=0.5, riskAdj_F=0.5, penalty=0.1
Resulting thresholds: min_raw_score=1.00, min_risk_adj_score=50.0
Generated Portfolio:
  Name: 2025-04-17_portf_rawF_0.5_riskAdjF_0.5_pen_0.1
  Number of stocks: 33

Running combination 3/320: raw_F=0.5, riskAdj_F=0.5, penalty=0.2
Resulting thresholds: min_raw_score=1.00, min_risk_adj_score=50.0
Generated Portfolio:
  Name: 2025-04-17_portf_rawF_0.5_riskAdjF_0.5_pen_0.2
  Number of stocks: 33

Running combination 4/320: raw_F=0.5, riskAdj_

In [206]:
all_portfolios

{'2025-04-17_portf_rawF_0.5_riskAdjF_0.5_pen_0.0': {'parameters': {'raw_score_scale_factor': np.float64(0.5),
   'risk_adj_score_scale_factor': np.float64(0.5),
   'penalty_IntraCluster_Corr': np.float64(0.0),
   'min_raw_score': np.float64(1.0),
   'min_risk_adj_score': np.float64(50.0),
   'select_top_n_clusters': 60,
   'max_selection_per_cluster': 2,
   'min_cluster_size': 3,
   'date': '2025-04-17'},
  'selected_stocks':         Weight
  Ticker        
  UGI     0.0115
  IBN     0.0201
  HDB     0.0175
  ABEV    0.0083
  IGF     0.0111
  IDV     0.0096
  SRAD    0.0106
  CPRT    0.0082
  IAGG    0.1163
  VUSB    0.3008
  SPSB    0.1801
  SGOL    0.0201
  GLDM    0.0199
  KR      0.0109
  DG      0.0088
  SBAC    0.0102
  CCI     0.0084
  COR     0.0154
  EXC     0.0128
  PPC     0.0172
  BTI     0.0100
  NOC     0.0116
  PM      0.0161
  ENB     0.0124
  TIGO    0.0138
  TEF     0.0131
  WCN     0.0107
  CCEP    0.0088
  ATO     0.0174
  NGG     0.0147
  CTRE    0.0141
  HRB     0

### AI Code


In [207]:
import pandas as pd
import numpy as np
import sys # Needed for sys.exit
import traceback # Import traceback for detailed error printing
import os

# --- Check for Parquet dependency ---
try:
    import pyarrow
    # Optional: Check for zstandard explicitly if needed, often comes with pyarrow
    # import zstandard
except ImportError:
    print("ERROR: 'pyarrow' library not found. Please install it (e.g., 'pip install pyarrow' or 'pip install pyarrow zstandard') to use Parquet format.")
    sys.exit("Exiting due to missing dependency.")


# --- Parameters ---
ADJ_CLOSE_COL = 'Adj Close'
POTENTIAL_TICKER_COLS = ['Symbol', 'Ticker']
POTENTIAL_DATE_COL = 'Date'
# --- Modified: Filenames now use .parquet extension ---
FINAL_RETURNS_FILENAME = 'portfolio_returns_final.parquet'
FINAL_SUMMARY_FILENAME = 'portfolio_factor_performance_final.parquet'
# --- Parameter: Control if final results are saved to file ---
SAVE_RESULTS_TO_FILE = True

print("--- Portfolio Performance Calculation Script ---")

# Assume df_OHLCV and all_portfolios are pre-loaded/defined before this script runs
# Example placeholders (replace with your actual loading logic):
# df_OHLCV = pd.read_csv('your_ohlcv_data.csv', index_col=[0,1], parse_dates=True) # Example
# all_portfolios = {'portfolio_1': {'parameters': {...}, 'selected_stocks': df_stocks1}, ...} # Example

# =============================================================================
# 1. Data and Index Preparation (NO CHANGE HERE)
# =============================================================================
print("\nChecking and preparing df_OHLCV index structure...")
ticker_level_name = None
date_level_name = None
try:
    # Check if df_OHLCV exists and is a DataFrame before modification
    if 'df_OHLCV' not in locals() or not isinstance(df_OHLCV, pd.DataFrame):
        print("ERROR: df_OHLCV is not defined or not a DataFrame.")
        sys.exit("Exiting due to missing data.")

    # Check if df_OHLCV already has a suitable MultiIndex
    if isinstance(df_OHLCV.index, pd.MultiIndex) and len(df_OHLCV.index.levels) >= 2:
        actual_names = list(df_OHLCV.index.names)
        print(f"Detected MultiIndex with names: {actual_names}")
        ticker_level_name = actual_names[0] if actual_names[0] is not None else 0
        date_level_name = actual_names[1] if actual_names[1] is not None else 1
        if isinstance(ticker_level_name, int) or isinstance(date_level_name, int):
             print(f"Warning: Using positional index levels ({ticker_level_name}, {date_level_name}) as names were missing.")
    else:
        print("Index is not a MultiIndex or has too few levels. Checking columns...")
        ticker_col_to_use = None
        for col in POTENTIAL_TICKER_COLS:
            if col in df_OHLCV.columns:
                ticker_col_to_use = col
                break
        date_col_to_use = POTENTIAL_DATE_COL if POTENTIAL_DATE_COL in df_OHLCV.columns else None

        if ticker_col_to_use and date_col_to_use:
            required_cols = [ticker_col_to_use, date_col_to_use]
            print(f"Attempting to set index using columns: {required_cols}...")
            df_OHLCV = df_OHLCV.set_index(required_cols)
            ticker_level_name = required_cols[0]
            date_level_name = required_cols[1]
            print(f"MultiIndex set successfully.")
        else:
            missing = []
            if not ticker_col_to_use: missing.extend(POTENTIAL_TICKER_COLS)
            if not date_col_to_use: missing.append(POTENTIAL_DATE_COL)
            print(f"ERROR: Cannot set index. Required columns for Ticker ({POTENTIAL_TICKER_COLS}) or Date ({POTENTIAL_DATE_COL}) not found.")
            print("df_OHLCV columns:", df_OHLCV.columns)
            sys.exit("Exiting due to incorrect df_OHLCV structure.")

    print(f"Using '{ticker_level_name}' for ticker level and '{date_level_name}' for date level.")

    if ADJ_CLOSE_COL not in df_OHLCV.columns:
        print(f"ERROR: Required price column '{ADJ_CLOSE_COL}' not found in df_OHLCV columns: {df_OHLCV.columns}")
        sys.exit("Exiting due to missing price column.")

    # Convert date level to datetime if it's not already
    date_level_values = df_OHLCV.index.get_level_values(date_level_name)
    if not pd.api.types.is_datetime64_any_dtype(date_level_values):
        print(f"Converting date level '{date_level_name}' to datetime objects...")
        try:
            # Get the current index levels and codes
            levels = list(df_OHLCV.index.levels)
            codes = list(df_OHLCV.index.codes)
            names = list(df_OHLCV.index.names)
            date_level_idx = names.index(date_level_name)

            # Convert only the date level
            levels[date_level_idx] = pd.to_datetime(levels[date_level_idx])

            # Reconstruct the index
            df_OHLCV.index = pd.MultiIndex(levels=levels, codes=codes, names=names)
            print("Date level converted.")
        except Exception as date_convert_e:
             print(f"ERROR: Could not convert date level '{date_level_name}' to datetime: {date_convert_e}")
             print("Attempting slower conversion using reset_index/set_index...")
             try:
                 original_columns = df_OHLCV.columns.tolist()
                 df_OHLCV = df_OHLCV.reset_index()
                 df_OHLCV[date_level_name] = pd.to_datetime(df_OHLCV[date_level_name])
                 df_OHLCV = df_OHLCV.set_index([ticker_level_name, date_level_name])
                 # Ensure original columns are preserved if reset_index added them
                 df_OHLCV = df_OHLCV[original_columns]
                 print("Date level converted using alternative method.")
             except Exception as fallback_e:
                 print(f"ERROR: Fallback date conversion also failed: {fallback_e}")
                 traceback.print_exc()
                 sys.exit("Exiting due to date conversion error.")


    if not df_OHLCV.index.is_monotonic_increasing:
         print("Sorting df_OHLCV index...")
         df_OHLCV = df_OHLCV.sort_index()
         print("Index sorted.")

except Exception as e:
    print(f"ERROR: An unexpected error occurred during index check/setup: {e}")
    traceback.print_exc()
    sys.exit("Exiting due to index setup error.")

# --- Get all unique tickers from df_OHLCV (NO CHANGE HERE) ---
all_known_tickers_set = set()
if 'df_OHLCV' in locals() and isinstance(df_OHLCV, pd.DataFrame) and ticker_level_name is not None:
    try:
        all_known_tickers = df_OHLCV.index.get_level_values(ticker_level_name).unique().tolist()
        all_known_tickers_set = set(all_known_tickers)
        print(f"\nFound {len(all_known_tickers_set):,} unique tickers in df_OHLCV data.")
        if not all_known_tickers_set:
             print("Warning: No tickers found in df_OHLCV index. Price fetching will likely fail.")
    except Exception as e:
        print(f"ERROR: Could not extract unique tickers from df_OHLCV index level '{ticker_level_name}': {e}")
        sys.exit("Exiting due to error extracting ticker list.")
else:
     print("Warning: df_OHLCV not available or ticker level name not set during initialization. Cannot pre-check ticker existence.")


# --- Extract Trading Dates (NO CHANGE HERE) ---
try:
    all_dates_in_data = df_OHLCV.index.get_level_values(date_level_name).unique()
    # Ensure conversion to datetime and sorting
    trading_dates = pd.DatetimeIndex(pd.to_datetime(all_dates_in_data)).sort_values()
    if not trading_dates.empty:
         print(f"Trading dates extracted using level '{date_level_name}'. Found {len(trading_dates)} unique dates (e.g., {trading_dates[0].date()} to {trading_dates[-1].date()}).")
    else:
         print(f"ERROR: No dates found in the index level '{date_level_name}'. Cannot proceed.")
         sys.exit("Exiting - Cannot determine trading dates.")
except (KeyError, IndexError) as e:
    print(f"ERROR: Could not find or access level '{date_level_name}' to extract trading dates: {e}")
    sys.exit("Exiting - Cannot determine trading dates.")
except Exception as e:
    print(f"ERROR: An unexpected error occurred extracting trading dates: {e}")
    traceback.print_exc()
    sys.exit("Exiting - Cannot determine trading dates.")

# =============================================================================
# 2. Helper Function for Trading Dates (NO CHANGE HERE)
# =============================================================================
def get_next_trading_date(current_date, sorted_trading_dates):
    """Finds the next trading date in the sorted list strictly after current_date."""
    if not isinstance(sorted_trading_dates, pd.DatetimeIndex) or not sorted_trading_dates.is_monotonic_increasing:
        try:
            # Ensure it's a sorted DatetimeIndex
            sorted_trading_dates = pd.DatetimeIndex(pd.to_datetime(sorted_trading_dates)).sort_values()
        except Exception:
             print("Error: Could not convert input to sorted DatetimeIndex in get_next_trading_date.")
             return None # Indicate error
    if sorted_trading_dates.empty:
        # print("Warning: Empty trading dates list provided to get_next_trading_date.")
        return None

    current_date = pd.Timestamp(current_date)
    try:
        # Use searchsorted to find the insertion point for the current_date
        # 'right' ensures we find the index *after* any occurrences of current_date
        loc = sorted_trading_dates.searchsorted(current_date, side='right')

        # If loc is within the bounds of the array, return that date
        if loc < len(sorted_trading_dates):
            return sorted_trading_dates[loc]
        else:
            # If loc is beyond the end, there's no next trading date
            # print(f"Info: No trading date found after {current_date.date()} in the provided list.")
            return None
    except Exception as e:
        print(f"Error during searchsorted in get_next_trading_date: {e}")
        return None # Indicate error

# =============================================================================
# 3. Portfolio Return Calculation Loop (NO CHANGE HERE)
# =============================================================================
# This list will hold results JUST from the CURRENT run
portfolio_returns_data_current_run = []
print("\nCalculating portfolio returns for the current run...")
index_mismatch_warning_shown = False

if 'all_portfolios' not in locals() or not isinstance(all_portfolios, dict):
    print("ERROR: 'all_portfolios' dictionary is not defined.")
    all_portfolios = {} # Define as empty to avoid errors below, though likely useless

if not all_portfolios:
    print("Warning: 'all_portfolios' dictionary is empty. No new returns to calculate.")
else:
    processed_count = 0
    skipped_count = 0
    error_count = 0
    for portf_name, portf_data in list(all_portfolios.items()): # Use list() for safe iteration if modifying dict
        try:
            # --- 3.1 Extract Parameters and Holdings ---
            params = portf_data.get('parameters')
            selected_stocks_df_orig = portf_data.get('selected_stocks') # DataFrame with Ticker/Symbol as index, 'Weight' column

            # Basic validation
            if not isinstance(params, dict) or not isinstance(selected_stocks_df_orig, pd.DataFrame) or 'Weight' not in selected_stocks_df_orig.columns:
                print(f"Warning: Skipping {portf_name} - Invalid structure or missing 'Weight' column in selected_stocks.")
                skipped_count += 1
                continue

            # Make a copy to avoid modifying the original dict entry directly
            selected_stocks_df = selected_stocks_df_orig.copy()

            # Ensure index has a name before comparison
            if selected_stocks_df.index.name is None:
                # potential_index_names = [name for name in POTENTIAL_TICKER_COLS if name in selected_stocks_df.columns or name.lower() in selected_stocks_df.index.name.lower() if selected_stocks_df.index.name else False]
                potential_index_names = [name for name in POTENTIAL_TICKER_COLS if
                         (name in selected_stocks_df.columns) or
                         (selected_stocks_df.index.name is not None and isinstance(selected_stocks_df.index.name, str) and name.lower() in selected_stocks_df.index.name.lower())]
                if potential_index_names:
                    selected_stocks_df.index.name = potential_index_names[0] # Use the first match
                else:
                    # Try setting from columns if Symbol/Ticker is a column
                    found_in_col = False
                    for col in POTENTIAL_TICKER_COLS:
                        if col in selected_stocks_df.columns:
                            try:
                                selected_stocks_df = selected_stocks_df.set_index(col)
                                found_in_col = True
                                break
                            except Exception as idx_err:
                                print(f"Warning: Could not set index from column {col} for {portf_name}: {idx_err}")
                    if not found_in_col:
                         print(f"Warning: Skipping {portf_name} - Could not determine Ticker index for selected_stocks.")
                         skipped_count += 1
                         continue


            portf_date_str = params.get('date')
            raw_factor = params.get('raw_score_scale_factor')
            risk_adj_factor = params.get('risk_adj_score_scale_factor')
            penalty_factor = params.get('penalty_IntraCluster_Corr')

            if portf_date_str is None or raw_factor is None or risk_adj_factor is None or penalty_factor is None:
                print(f"Warning: Skipping {portf_name} - Missing essential parameter (date, raw_factor, risk_adj_factor, or penalty_factor).")
                skipped_count += 1
                continue

            try:
                portf_date = pd.Timestamp(portf_date_str)
            except ValueError:
                print(f"Warning: Skipping {portf_name} - Invalid date format for parameter 'date': {portf_date_str}")
                skipped_count += 1
                continue


            if selected_stocks_df.empty:
                # print(f"Info: Skipping {portf_name} - Portfolio has no selected stocks.")
                skipped_count += 1
                continue

            # --- 3.2 Ensure Portfolio Index Matches Data Index Name ---
            if selected_stocks_df.index.name != ticker_level_name:
                if not index_mismatch_warning_shown:
                    print(f"Warning: Portfolio index name ('{selected_stocks_df.index.name}') mismatches data index ('{ticker_level_name}'). Renaming portfolio index for alignment.")
                    index_mismatch_warning_shown = True
                try:
                    selected_stocks_df.index.name = ticker_level_name
                except Exception as rename_e:
                    print(f"Error: Failed to rename portfolio index for {portf_name}: {rename_e}. Skipping.")
                    skipped_count += 1
                    continue

            original_portfolio_tickers = selected_stocks_df.index.tolist()
            if not original_portfolio_tickers: # Double check after potential renaming/copying
                 # print(f"Info: Skipping {portf_name} - Portfolio has no tickers after index handling.")
                 skipped_count += 1
                 continue

            # --- Check: Verify tickers exist in df_OHLCV's known tickers ---
            tickers_in_portfolio_set = set(original_portfolio_tickers)
            missing_from_ohlcv = tickers_in_portfolio_set - all_known_tickers_set # Find tickers in portfolio but not in OHLCV data

            if missing_from_ohlcv:
                # print(f"Warning for {portf_name}: The following tickers selected for the portfolio are completely missing from the historical price data (df_OHLCV) and will be dropped from calculation: {sorted(list(missing_from_ohlcv))}")
                # Filter the list of tickers to proceed with
                tickers_to_process = [t for t in original_portfolio_tickers if t in all_known_tickers_set]

                if not tickers_to_process:
                    # print(f"Info: Skipping {portf_name} - No tickers remaining after removing those completely missing from price data.")
                    skipped_count += 1
                    continue # Skip to the next portfolio

                # IMPORTANT: Filter the selected_stocks_df as well to keep weights consistent
                selected_stocks_df = selected_stocks_df.loc[tickers_to_process]
                original_portfolio_tickers = tickers_to_process # Update the list for downstream use
            # --- End of Check ---

            # --- 3.3 Determine Buy and Sell Dates ---
            buy_date = get_next_trading_date(portf_date, trading_dates)
            if buy_date is None:
                # print(f"Info: Skipping {portf_name} - Cannot find trading date after portfolio generation date {portf_date.date()}.")
                skipped_count += 1
                continue
            sell_date = get_next_trading_date(buy_date, trading_dates)
            if sell_date is None:
                # print(f"Info: Skipping {portf_name} - Cannot find trading date after buy date {buy_date.date()}.")
                skipped_count += 1
                continue
            required_dates = [buy_date, sell_date]

            # --- 3.4 Fetch Price Data & Filter Available Tickers ---
            # Use the potentially filtered 'original_portfolio_tickers'
            idx = pd.IndexSlice
            try:
                 # Use reindex for robustness against missing ticker/date combinations
                 multi_idx = pd.MultiIndex.from_product([original_portfolio_tickers, required_dates], names=[ticker_level_name, date_level_name])
                 potential_data = df_OHLCV.reindex(multi_idx) # This will have NaNs where data is missing

            except Exception as slice_error:
                 print(f"Error: Unexpected error during data slicing/reindexing for {portf_name}: {slice_error}. Skipping.")
                 skipped_count+=1
                 continue


            # Identify tickers that have *valid* data for BOTH required dates
            # Group by ticker and check count of non-NA Adj Close prices for the required dates
            prices_for_check = potential_data[[ADJ_CLOSE_COL]].dropna() # Drop rows with NaN Adj Close
            if prices_for_check.empty:
                # print(f"Info: Skipping {portf_name} - No valid price data found for any selected ticker on required dates {buy_date.date()}, {sell_date.date()}.")
                skipped_count += 1
                continue

            # Count how many of the required dates each ticker has valid data for
            ticker_date_counts = prices_for_check.index.get_level_values(ticker_level_name).value_counts()

            # Keep only tickers that have data for *all* required dates (i.e., count == 2)
            available_tickers = ticker_date_counts[ticker_date_counts == len(required_dates)].index.tolist()

            if not available_tickers:
                # print(f"Info: Skipping {portf_name} - No stocks found with non-NaN '{ADJ_CLOSE_COL}' for BOTH Buy({buy_date.date()}) and Sell({sell_date.date()}) dates.")
                skipped_count += 1
                continue

            # --- 3.5 Process Prices for Available Tickers ---
            try:
                # Select 'Adj Close' for the available tickers and dates from potential_data
                idx_slice = pd.IndexSlice[available_tickers, required_dates]
                # Use .loc on potential_data which is already filtered by dates/tickers
                prices = potential_data.loc[idx_slice, ADJ_CLOSE_COL]

                prices_unstacked = prices.unstack(level=date_level_name)
                prices_unstacked = prices_unstacked.rename(columns={buy_date: 'Buy Price', sell_date: 'Sell Price'})

                # Final check for NaNs that might have crept in (though reindex/dropna should handle most)
                prices_unstacked = prices_unstacked.dropna()

                if prices_unstacked.empty:
                    # print(f"Info: Skipping {portf_name} - Price data became empty after unstacking/final dropna.")
                    skipped_count += 1
                    continue

                final_available_tickers = prices_unstacked.index.tolist()

            except KeyError as e:
                 print(f"Error: KeyError during price processing for {portf_name}: {e}. Index: {idx_slice}. Check slicing or column '{ADJ_CLOSE_COL}'. Skipping.")
                 skipped_count += 1
                 continue
            except Exception as e:
                 print(f"Error: Unexpected error during price processing for {portf_name}: {e}. Skipping.")
                 traceback.print_exc()
                 skipped_count += 1
                 continue

            # --- 3.6 Calculate Portfolio Return ---
            # Align weights using the 'selected_stocks_df' (already filtered for OHLCV presence)
            # Index it with the tickers that *actually have price data* for the calculation period.
            aligned_weights = selected_stocks_df.loc[final_available_tickers, 'Weight']

            # ---> Weight Handling (Normalization) <---
            weight_sum = aligned_weights.sum()
            if abs(weight_sum) < 1e-9: # Use absolute value for check
                 # print(f"Info: Skipping {portf_name} - Sum of weights for available stocks is zero.")
                 skipped_count += 1
                 continue
            normalized_weights = aligned_weights / weight_sum
            # ---> End of Weight Handling <---

            buy_prices = prices_unstacked.loc[final_available_tickers, 'Buy Price']
            sell_prices = prices_unstacked.loc[final_available_tickers, 'Sell Price']

            # --- Handle potential zero buy prices ---
            zero_price_mask = (buy_prices.abs() < 1e-9)
            if zero_price_mask.any():
                num_zero_prices = zero_price_mask.sum()
                # print(f"Warning for {portf_name}: Found {num_zero_prices} stock(s) with near-zero buy price. They will be excluded from return calculation.")
                valid_price_mask = ~zero_price_mask
                if not valid_price_mask.any(): # Check if *any* are True
                    # print(f"Info: Skipping {portf_name} - All remaining stocks had zero buy price.")
                    skipped_count += 1
                    continue

                # Filter prices and weights
                buy_prices = buy_prices.loc[valid_price_mask]
                sell_prices = sell_prices.loc[valid_price_mask]
                normalized_weights = normalized_weights.loc[valid_price_mask] # Align weights again
                final_available_tickers = buy_prices.index.tolist() # Update ticker list

                # Re-normalize weights after removing zero-price stocks
                weight_sum = normalized_weights.sum()
                if abs(weight_sum) < 1e-9:
                    # print(f"Info: Skipping {portf_name} - Sum of weights became zero after removing zero-price stocks.")
                    skipped_count += 1
                    continue
                normalized_weights = normalized_weights / weight_sum # Renormalize again

            # --- Final check for NaN/Inf before calculation ---
            if buy_prices.isnull().any() or sell_prices.isnull().any() or normalized_weights.isnull().any() or \
               np.isinf(buy_prices).any() or np.isinf(sell_prices).any() or np.isinf(normalized_weights).any():
                print(f"Warning: Skipping {portf_name} - Found NaN or Inf in final arrays (prices or weights) before calculation.")
                skipped_count += 1
                continue

            # --- Calculate weighted return ---
            individual_returns = (sell_prices / buy_prices) - 1 # Division handles potential remaining zeros gracefully -> inf
            individual_returns = individual_returns.replace([np.inf, -np.inf], np.nan).fillna(0) # Replace inf/nan from division with 0 return

            portfolio_return = (individual_returns * normalized_weights).sum()

             # --- Final check on the calculated return ---
            if pd.isna(portfolio_return) or np.isinf(portfolio_return):
                print(f"Warning: Skipping {portf_name} - Final calculated portfolio return is NaN or Inf.")
                skipped_count += 1
                continue

            # --- 3.7 Store Results for Current Run ---
            portfolio_returns_data_current_run.append({
                'portfolio_name': portf_name,
                'generation_date': portf_date,
                'buy_date': buy_date,
                'sell_date': sell_date,
                'raw_factor': raw_factor,
                'risk_adj_factor': risk_adj_factor,
                'penalty_factor': penalty_factor,
                'portfolio_return': portfolio_return,
                # num_stocks_selected: Count from original selection (before any filtering)
                'num_stocks_selected': len(tickers_in_portfolio_set),
                # num_stocks_in_ohlcv: Count after removing those completely missing from OHLCV
                'num_stocks_in_ohlcv': len(selected_stocks_df.index), # Use the df potentially filtered for OHLCV presence
                # num_stocks_calc: Count actually used in calculation (passed all filters: OHLCV, dates, zero price)
                'num_stocks_calc': len(final_available_tickers)
            })
            processed_count += 1

        except Exception as e:
            print(f"FATAL Error processing portfolio {portf_name}: {e}")
            traceback.print_exc()
            error_count += 1
            # Decide whether to continue or stop the loop
            # continue # Continue to next portfolio even on fatal error for one

    # --- Loop Finished ---
    print(f"\nCurrent run portfolio return calculation finished.")
    print(f"Successfully processed: {processed_count}")
    print(f"Skipped (data issues/filters): {skipped_count}")
    print(f"Errors during processing: {error_count}")

# =============================================================================
# 4. Load Historical Data (if exists), Combine, Clean, and Analyze
# =============================================================================
print("\n--- Loading Historical Data, Combining with Current Run, and Cleaning ---")

# Initialize DataFrames to hold all data (historical + current) and the summary
all_returns_df = pd.DataFrame()
performance_summary_df = pd.DataFrame() # Initialize summary DataFrame

# --- Try to load existing historical data ---
historical_returns_df = pd.DataFrame() # Initialize empty df for historical data
if os.path.exists(FINAL_RETURNS_FILENAME):
    try:
        print(f"Loading historical data from {FINAL_RETURNS_FILENAME}...")
        # --- *** MODIFIED: Use read_parquet *** ---
        historical_returns_df = pd.read_parquet(FINAL_RETURNS_FILENAME, engine='pyarrow')
        # --- *** END MODIFICATION *** ---

        # Basic check for essential columns (can be expanded)
        # Parquet usually preserves types well, including dates, but checks are still good
        expected_cols = ['generation_date', 'buy_date', 'sell_date', 'raw_factor', 'risk_adj_factor', 'penalty_factor', 'portfolio_return']
        if all(col in historical_returns_df.columns for col in expected_cols):
            print(f"Loaded {len(historical_returns_df)} historical records.")
            # Optional: Verify date types if needed, though pyarrow usually handles this
            # for col in ['generation_date', 'buy_date', 'sell_date']:
            #     if not pd.api.types.is_datetime64_any_dtype(historical_returns_df[col]):
            #         print(f"Warning: Column {col} in loaded Parquet file is not datetime. Attempting conversion.")
            #         historical_returns_df[col] = pd.to_datetime(historical_returns_df[col], errors='coerce')

        else:
            print(f"Warning: Historical data file {FINAL_RETURNS_FILENAME} is missing expected columns. Ignoring historical data.")
            historical_returns_df = pd.DataFrame() # Reset to empty if format is wrong
            # Optional: Rename/backup the bad file
            # try:
            #     os.rename(FINAL_RETURNS_FILENAME, FINAL_RETURNS_FILENAME + f".bad_format_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}")
            # except OSError as ren_err:
            #     print(f"Could not rename bad format file: {ren_err}")

    except FileNotFoundError:
         # This case is handled by os.path.exists, but included for completeness
         print(f"Historical data file {FINAL_RETURNS_FILENAME} not found.")
         historical_returns_df = pd.DataFrame()
    # except pyarrow.lib.ArrowIOError as arrow_io_err: # More specific error for Parquet I/O
    #      print(f"Error reading Parquet file {FINAL_RETURNS_FILENAME}: {arrow_io_err}")
    #      print("Warning: File might be corrupted or not a valid Parquet file. Proceeding without historical data.")
    #      historical_returns_df = pd.DataFrame()
    except Exception as e:
        print(f"Error loading historical data from {FINAL_RETURNS_FILENAME}: {e}")
        traceback.print_exc()
        print("Warning: Proceeding without historical data.")
        historical_returns_df = pd.DataFrame()
else:
    print(f"Historical data file {FINAL_RETURNS_FILENAME} not found. Starting fresh.")

# --- Combine with current run's data (NO CHANGE HERE) ---
current_returns_df = pd.DataFrame() # Initialize empty df for current run
if portfolio_returns_data_current_run:
    current_returns_df = pd.DataFrame(portfolio_returns_data_current_run)
    print(f"Created DataFrame with {len(current_returns_df)} new records from this run.")

    # Ensure date columns are datetime objects in the new data (should be already, but good practice)
    for col in ['generation_date', 'buy_date', 'sell_date']:
         if col in current_returns_df.columns:
              current_returns_df[col] = pd.to_datetime(current_returns_df[col])
else:
    print("No new portfolio returns were calculated in this run.")


# --- Concatenate historical and current data (NO CHANGE HERE) ---
if not historical_returns_df.empty or not current_returns_df.empty:
    all_returns_df = pd.concat([historical_returns_df, current_returns_df], ignore_index=True)
    print(f"Combined data contains {len(all_returns_df)} records (before deduplication).")

    # --- Data Cleaning: Remove Duplicates (NO CHANGE HERE) ---
    key_cols = ['generation_date', 'buy_date', 'sell_date', 'raw_factor', 'risk_adj_factor', 'penalty_factor']
    if all(col in all_returns_df.columns for col in key_cols):
        initial_len = len(all_returns_df)
        all_returns_df = all_returns_df.drop_duplicates(subset=key_cols, keep='last')
        duplicates_removed = initial_len - len(all_returns_df)
        if duplicates_removed > 0:
            print(f"Removed {duplicates_removed} duplicate records based on key columns: {key_cols}.")
        else:
            print("No duplicate records found based on key columns.")
    else:
        print(f"Warning: Cannot perform duplicate check. Not all key columns {key_cols} found in combined data.")
        print(f"Columns present: {all_returns_df.columns.tolist()}")

else:
    print("No historical or current return data available to process.")


# --- Perform analysis ONLY if there's combined data (NO CHANGE HERE) ---
if not all_returns_df.empty:
    print("\n--- Performing Analysis on Combined and Cleaned Data ---")
    # Ensure correct data types before grouping (especially factors and return)
    try:
        numeric_cols = ['raw_factor', 'risk_adj_factor', 'penalty_factor', 'portfolio_return']
        for col in numeric_cols:
            if col in all_returns_df.columns:
                all_returns_df[col] = pd.to_numeric(all_returns_df[col], errors='coerce') # Coerce errors to NaN

        # Optional: Check for NaNs introduced by coercion
        nas_after_conversion = all_returns_df[numeric_cols].isnull().sum()
        if nas_after_conversion.sum() > 0:
            print("Warning: Found NaNs after converting numeric columns:")
            print(nas_after_conversion[nas_after_conversion > 0])

    except Exception as e:
        print(f"Error converting factor/return columns to numeric: {e}. Analysis might be affected.")


    # Sort for easier viewing and consistent grouping (optional, but good practice)
    grouping_factors = ['raw_factor', 'risk_adj_factor', 'penalty_factor']
    # Ensure grouping factors exist before sorting/grouping
    valid_grouping_factors = [f for f in grouping_factors if f in all_returns_df.columns]
    if len(valid_grouping_factors) != len(grouping_factors):
        print(f"Warning: Not all grouping factors {grouping_factors} exist in the data. Using: {valid_grouping_factors}")
    grouping_factors = valid_grouping_factors # Use only valid factors

    if not grouping_factors:
        print("Error: No valid grouping factors available for analysis. Skipping summary.")
    else:
        try:
            # Sort before grouping
            all_returns_df = all_returns_df.sort_values(by=['generation_date'] + grouping_factors)

            print(f"\nTotal unique return instances after cleaning: {len(all_returns_df)}")
            # print("Sample of cleaned combined data:") # Optional
            # print(all_returns_df.head())

            # --- Group by factor combinations to analyze performance ---
            print("\n--- Performance Analysis by Factor Combination (Based on Cleaned Data) ---")

            # Define aggregation functions
            def count_valid(x):
                return x.dropna().count()
            def std_dev(x):
                 valid_count = x.dropna().count()
                 if valid_count >= 2:
                      return x.std(ddof=1) # Use sample std dev (ddof=1)
                 else:
                      return np.nan
            def sharpe_ratio(x, risk_free_rate=0, periods_per_year=252):
                 valid_count = x.dropna().count()
                 if valid_count < 2:
                      return np.nan
                 mean_return = x.mean()
                 std_return = x.std(ddof=1)
                 if std_return is None or pd.isna(std_return) or abs(std_return) < 1e-9:
                      return np.nan
                 sharpe = (mean_return - risk_free_rate) / std_return
                 return sharpe * np.sqrt(periods_per_year)

            # Group by all specified factors and aggregate portfolio returns
            performance_summary_df = all_returns_df.groupby(grouping_factors)['portfolio_return'].agg(
                mean='mean',
                median='median',
                std=std_dev,
                count=count_valid,
                sharpe=sharpe_ratio # Use the defined function
            ).reset_index() # Reset index to make factors columns again

            # --- Calculate additional metrics ---
            if 'portfolio_return' in all_returns_df.columns:
                 win_rate = all_returns_df[all_returns_df['portfolio_return'] > 0].groupby(grouping_factors).size() / \
                            all_returns_df.groupby(grouping_factors).size()
                 performance_summary_df = pd.merge(performance_summary_df, win_rate.rename('win_rate').reset_index(), on=grouping_factors, how='left')
                 performance_summary_df['win_rate'] = performance_summary_df['win_rate'].fillna(0) # Fill NaN if no trades for a combo

            # Sort by a chosen metric (e.g., Sharpe ratio, mean return)
            sort_metric = 'sharpe' if 'sharpe' in performance_summary_df.columns else 'mean'
            if sort_metric in performance_summary_df.columns:
                performance_summary_df = performance_summary_df.sort_values(by=sort_metric, ascending=False, na_position='last')
                print(f"\nPerformance per Factor Combination (sorted by {sort_metric}):")
            else:
                 print(f"\nPerformance per Factor Combination (unsorted):")

            with pd.option_context('display.float_format', '{:.4f}'.format, 'display.max_rows', 200):
                print(performance_summary_df)

            # --- Analyze the number of stocks used ---
            stock_count_cols = ['num_stocks_selected', 'num_stocks_in_ohlcv', 'num_stocks_calc']
            valid_stock_cols = [c for c in stock_count_cols if c in all_returns_df.columns]
            if valid_stock_cols:
                 stock_count_summary = all_returns_df.groupby(grouping_factors)[valid_stock_cols].agg(['mean', 'median', 'min', 'max'])
                 print("\nAverage/Median/Min/Max Stocks per Factor Combination (Cleaned Data):")
                 with pd.option_context('display.float_format', '{:.1f}'.format):
                      print(stock_count_summary)

        except Exception as analysis_err:
            print(f"\nERROR during performance analysis: {analysis_err}")
            traceback.print_exc()
            # Ensure performance_summary_df is empty DataFrame if analysis fails
            performance_summary_df = pd.DataFrame()

else:
    print("\nNo portfolio returns data available (neither historical nor current run) to analyze after cleaning.")
    # Ensure performance_summary_df is defined as empty
    performance_summary_df = pd.DataFrame()

# =============================================================================
# 5. Optionally Save Final Results (Overwrite Mode)
# =============================================================================
if SAVE_RESULTS_TO_FILE:
    print("\n--- Saving Final Results to Parquet ---")
    # --- Save the combined, cleaned returns data (OVERWRITE) ---
    if not all_returns_df.empty:
        try:
            # --- *** MODIFIED: Use to_parquet *** ---
            all_returns_df.to_parquet(
                FINAL_RETURNS_FILENAME,
                engine='pyarrow',
                compression='zstd', # Use zstd compression
                index=False         # Do not write the DataFrame index
            )
            # --- *** END MODIFICATION *** ---
            print(f"Saved combined and cleaned returns ({len(all_returns_df)} records) to {FINAL_RETURNS_FILENAME}")
        except Exception as e:
            print(f"Error saving final returns data to {FINAL_RETURNS_FILENAME}: {e}")
            traceback.print_exc() # Print detailed error
    else:
        print(f"No combined returns data to save to {FINAL_RETURNS_FILENAME}.")
        # Optional: remove existing file if no data to save
        # if os.path.exists(FINAL_RETURNS_FILENAME):
        #     try:
        #         os.remove(FINAL_RETURNS_FILENAME)
        #         print(f"Removed existing empty file: {FINAL_RETURNS_FILENAME}")
        #     except OSError as rem_err:
        #          print(f"Could not remove existing file {FINAL_RETURNS_FILENAME}: {rem_err}")


    # --- Save the performance summary (OVERWRITE) ---
    if not performance_summary_df.empty:
        try:
            # --- *** MODIFIED: Use to_parquet *** ---
            performance_summary_df.to_parquet(
                FINAL_SUMMARY_FILENAME,
                engine='pyarrow',
                compression='zstd', # Use zstd compression
                index=False         # Do not write the DataFrame index
            )
            # --- *** END MODIFICATION *** ---
            print(f"Saved performance summary ({len(performance_summary_df)} factor combinations) to {FINAL_SUMMARY_FILENAME}")
        except Exception as e:
            print(f"Error saving performance summary to {FINAL_SUMMARY_FILENAME}: {e}")
            traceback.print_exc() # Print detailed error
    else:
        print(f"No performance summary data to save to {FINAL_SUMMARY_FILENAME}.")
        # Optional: remove existing file if no data to save
        # if os.path.exists(FINAL_SUMMARY_FILENAME):
        #      try:
        #         os.remove(FINAL_SUMMARY_FILENAME)
        #         print(f"Removed existing empty file: {FINAL_SUMMARY_FILENAME}")
        #      except OSError as rem_err:
        #          print(f"Could not remove existing file {FINAL_SUMMARY_FILENAME}: {rem_err}")

else:
    print("\n--- Skipping Saving Results to File (SAVE_RESULTS_TO_FILE=False) ---")
    print(f"Returns DataFrame ('all_returns_df') contains {len(all_returns_df)} rows.")
    print(f"Summary DataFrame ('performance_summary_df') contains {len(performance_summary_df)} rows.")


print("\n--- Script Finished ---")

--- Portfolio Performance Calculation Script ---

Checking and preparing df_OHLCV index structure...
Detected MultiIndex with names: ['Symbol', 'Date']
Using 'Symbol' for ticker level and 'Date' for date level.
Sorting df_OHLCV index...
Index sorted.

Found 1,538 unique tickers in df_OHLCV data.
Trading dates extracted using level 'Date'. Found 251 unique dates (e.g., 2024-04-18 to 2025-04-17).

Calculating portfolio returns for the current run...

Current run portfolio return calculation finished.
Successfully processed: 0
Skipped (data issues/filters): 320
Errors during processing: 0

--- Loading Historical Data, Combining with Current Run, and Cleaning ---
Loading historical data from portfolio_returns_final.parquet...
Loaded 3495 historical records.
No new portfolio returns were calculated in this run.
Combined data contains 3495 records (before deduplication).
No duplicate records found based on key columns.

--- Performing Analysis on Combined and Cleaned Data ---

Total unique r

In [208]:
portfolio_returns_df = pd.read_parquet('portfolio_returns_final.parquet', engine='pyarrow')
portfolio_returns_df

,portfolio_name,generation_date,buy_date,sell_date,raw_factor,risk_adj_factor,penalty_factor,portfolio_return,num_stocks_selected,num_stocks_in_ohlcv,num_stocks_calc
0,2025-04-01_portf_rawF_0.5_riskAdjF_0.5_pen_0.0,2025-04-01,2025-04-02,2025-04-03,0.5000,0.5000,0.0000,0.0004,36,35,35
1,2025-04-01_portf_rawF_0.5_riskAdjF_0.5_pen_0.1,2025-04-01,2025-04-02,2025-04-03,0.5000,0.5000,0.1000,0.0004,36,35,35
2,2025-04-01_portf_rawF_0.5_riskAdjF_0.5_pen_0.2,2025-04-01,2025-04-02,2025-04-03,0.5000,0.5000,0.2000,0.0004,36,35,35
3,2025-04-01_portf_rawF_0.5_riskAdjF_0.5_pen_0.3,2025-04-01,2025-04-02,2025-04-03,0.5000,0.5000,0.3000,0.0004,36,35,35
4,2025-04-01_portf_rawF_0.5_riskAdjF_0.5_pen_0.4,2025-04-01,2025-04-02,2025-04-03,0.5000,0.5000,0.4000,0.0004,36,35,35
...,...,...,...,...,...,...,...,...,...,...,...
3490,2025-04-15_portf_rawF_1.2_riskAdjF_1.2_pen_0.0,2025-04-15,2025-04-16,2025-04-17,1.2000,1.2000,0.0000,0.0002,1,1,1
3491,2025-04-15_portf_rawF_1.2_riskAdjF_1.2_pen_0.1,2025-04-15,2025-04-16,2025-04-17,1.2000,1.2000,0.1000,0.0002,1,1,1
3492,2025-04-15_portf_rawF_1.2_riskAdjF_1.2_pen_0.2,2025-04-15,2025-04-16,2025-04-17,1.2000,1.2000,0.2000,0.0002,1,1,1
3493,2025-04-15_portf_rawF_1.2_riskAdjF_1.2_pen_0.3,2025-04-15,2025-04-16,2025-04-17,1.2000,1.2000,0.3000,0.0002,1,1,1


In [209]:
portfolio_factor_performance_final = pd.read_parquet('portfolio_factor_performance_final.parquet', engine='pyarrow')
portfolio_factor_performance_final

,raw_factor,risk_adj_factor,penalty_factor,mean,median,std,count,sharpe,win_rate
0,0.5000,0.7000,0.4000,-0.0036,0.0003,0.0219,11,-2.6076,0.7273
1,0.5000,0.7000,0.0000,-0.0036,0.0003,0.0219,11,-2.6076,0.7273
2,0.5000,0.7000,0.3000,-0.0036,0.0003,0.0219,11,-2.6076,0.7273
3,0.5000,0.7000,0.2000,-0.0036,0.0003,0.0219,11,-2.6076,0.7273
4,0.5000,0.7000,0.1000,-0.0036,0.0003,0.0219,11,-2.6076,0.7273
...,...,...,...,...,...,...,...,...,...
315,1.1000,0.5000,0.0000,-0.0147,-0.0025,0.0352,11,-6.6351,0.3636
316,1.1000,0.5000,0.1000,-0.0147,-0.0025,0.0352,11,-6.6351,0.3636
317,1.1000,0.5000,0.2000,-0.0147,-0.0025,0.0352,11,-6.6351,0.3636
318,1.1000,0.5000,0.3000,-0.0147,-0.0025,0.0352,11,-6.6351,0.3636


In [210]:
sorted_performance = portfolio_factor_performance_final.sort_values(by='sharpe', ascending=False)
display(sorted_performance)


,raw_factor,risk_adj_factor,penalty_factor,mean,median,std,count,sharpe,win_rate
0,0.5000,0.7000,0.4000,-0.0036,0.0003,0.0219,11,-2.6076,0.7273
2,0.5000,0.7000,0.3000,-0.0036,0.0003,0.0219,11,-2.6076,0.7273
3,0.5000,0.7000,0.2000,-0.0036,0.0003,0.0219,11,-2.6076,0.7273
4,0.5000,0.7000,0.1000,-0.0036,0.0003,0.0219,11,-2.6076,0.7273
1,0.5000,0.7000,0.0000,-0.0036,0.0003,0.0219,11,-2.6076,0.7273
...,...,...,...,...,...,...,...,...,...
315,1.1000,0.5000,0.0000,-0.0147,-0.0025,0.0352,11,-6.6351,0.3636
316,1.1000,0.5000,0.1000,-0.0147,-0.0025,0.0352,11,-6.6351,0.3636
317,1.1000,0.5000,0.2000,-0.0147,-0.0025,0.0352,11,-6.6351,0.3636
318,1.1000,0.5000,0.3000,-0.0147,-0.0025,0.0352,11,-6.6351,0.3636


In [211]:
# # =============================================================================
# # 4b. Time-Based Performance Analysis (Example: By Year)
# # =============================================================================
# if not all_returns_df.empty:
#     print("\n--- Performance Analysis by Factor Combination AND Year ---")

#     # Ensure 'sell_date' is datetime and extract the year
#     try:
#         all_returns_df['sell_year'] = pd.to_datetime(all_returns_df['sell_date']).dt.year
        
#         # Define grouping factors including the year
#         time_grouping_factors = ['sell_year'] + grouping_factors # grouping_factors was ['raw_factor', 'risk_adj_factor', 'penalty_factor']

#         # Group by year and factors
#         performance_by_year = all_returns_df.groupby(time_grouping_factors)['portfolio_return'].agg(
#             mean='mean',
#             median='median',
#             std=lambda x: x.std(ddof=0) if pd.notna(x).sum() > 1 else np.nan,
#             count=lambda x: pd.notna(x).sum()
#             # Add other metrics like Sharpe, win rate per year if desired
#         ).reset_index()

#         # Sort for better readability (e.g., by year, then by mean return)
#         performance_by_year = performance_by_year.sort_values(by=['sell_year', 'mean'], ascending=[True, False])

#         print("\nPerformance per Factor Combination per Year (sorted by year, then mean return):")
#         with pd.option_context('display.float_format', '{:.4f}'.format, 'display.max_rows', 500): # Show more rows
#             print(performance_by_year)

#         # You could save this to a separate CSV if needed
#         # performance_by_year.to_csv('portfolio_factor_performance_by_year.csv', index=False)

#     except Exception as e:
#         print(f"Error during yearly performance analysis: {e}")
#         traceback.print_exc()
# else:
    # print("\nNo data available for yearly performance analysis.")